In [74]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import pytz
import datetime

nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\hprus\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [75]:
#Laod the datasets
apps_df = pd.read_csv("C:\\Users\\hprus\\Downloads\\Play Store Data.csv")
reviews_df = pd.read_csv("C:\\Users\\hprus\\Downloads\\User Reviews.csv")


In [76]:

apps_df.head()



,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [77]:
reviews_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [78]:
print("\n Apps")
print(apps_df.isnull().sum())

print("\n Reviews")
print(reviews_df.isnull().sum())


 Apps
App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

 Reviews
App                           0
Translated_Review         26868
Sentiment                 26863
Sentiment_Polarity        26863
Sentiment_Subjectivity    26863
dtype: int64


In [79]:
print("\n Apps")
print(apps_df.dtypes)

print("\n Reviews")
print(reviews_df.dtypes)


 Apps
App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

 Reviews
App                        object
Translated_Review          object
Sentiment                  object
Sentiment_Polarity        float64
Sentiment_Subjectivity    float64
dtype: object


In [80]:
print("\n Apps")
print(apps_df.info())

print("\n Reviews")
print(reviews_df.info())


 Apps
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB
None

 Reviews
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64295 entries, 0 to 64294
Data columns (total 5 columns):
 #   Column                  Non-

In [81]:
print("\n Apps")
print(apps_df.describe())

print("\n Reviews")
print(reviews_df.describe())


 Apps
            Rating
count  9367.000000
mean      4.193338
std       0.537431
min       1.000000
25%       4.000000
50%       4.300000
75%       4.500000
max      19.000000

 Reviews
       Sentiment_Polarity  Sentiment_Subjectivity
count        37432.000000            37432.000000
mean             0.182146                0.492704
std              0.351301                0.259949
min             -1.000000                0.000000
25%              0.000000                0.357143
50%              0.150000                0.514286
75%              0.400000                0.650000
max              1.000000                1.000000


In [82]:
# Convert Installs
# Clean Installs column safely
apps_df["Installs"] = (
    apps_df["Installs"]
    .astype(str)                               # force to string
    .str.replace(r"[+,]", "", regex=True)      # remove '+' and ','
    .replace({"<NA>": None, "nan": None, "NaN": None, "": None})  # handle weird NA strings
)


# Convert Reviews
apps_df["Reviews"] = pd.to_numeric(apps_df["Reviews"], errors="coerce")

# Convert Last Updated
apps_df["Last Updated"] = pd.to_datetime(apps_df["Last Updated"], errors="coerce")

In [83]:
apps_df.drop_duplicates(inplace=True)
apps_df = apps_df.dropna(subset=["App", "Category", "Rating", "Size", "Reviews", "Installs", "Last Updated"])


In [84]:
# Convert Size → MB
def size_to_mb(size):
    size = str(size).strip()
    if size.endswith("M"):
        return float(size.replace("M", ""))
    elif size.endswith("k"):
        return float(size.replace("k", "")) / 1024
    else:
        try:
            return float(size)
        except:
            return None

apps_df["Size_MB"] = apps_df["Size"].apply(size_to_mb)


In [85]:
apps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_MB
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159.0,19M,10000,Free,0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,19.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967.0,14M,500000,Free,0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,14.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510.0,8.7M,5000000,Free,0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,8.7
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644.0,25M,50000000,Free,0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,25.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967.0,2.8M,100000,Free,0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,2.8


In [86]:
reviews_df.drop_duplicates(inplace=True)
reviews_df = reviews_df.dropna(subset=["App", "Translated_Review"])

In [87]:
reviews_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000
5,10 Best Foods for You,Best way,Positive,1.00,0.300000


In [88]:
#Merging the datasets
merged_df = pd.merge(reviews_df, apps_df, on="App", how="inner")
merged_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_MB
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333,HEALTH_AND_FITNESS,4.0,2490.0,3.8M,500000,Free,0,Everyone 10+,Health & Fitness,2017-02-17,1.9,2.3.3 and up,3.8
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462,HEALTH_AND_FITNESS,4.0,2490.0,3.8M,500000,Free,0,Everyone 10+,Health & Fitness,2017-02-17,1.9,2.3.3 and up,3.8
2,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000,HEALTH_AND_FITNESS,4.0,2490.0,3.8M,500000,Free,0,Everyone 10+,Health & Fitness,2017-02-17,1.9,2.3.3 and up,3.8
3,10 Best Foods for You,Best idea us,Positive,1.00,0.300000,HEALTH_AND_FITNESS,4.0,2490.0,3.8M,500000,Free,0,Everyone 10+,Health & Fitness,2017-02-17,1.9,2.3.3 and up,3.8
4,10 Best Foods for You,Best way,Positive,1.00,0.300000,HEALTH_AND_FITNESS,4.0,2490.0,3.8M,500000,Free,0,Everyone 10+,Health & Fitness,2017-02-17,1.9,2.3.3 and up,3.8


In [89]:
#Applying the filters
filtered = merged_df[
    (merged_df["Rating"] >= 4.0) &
    (merged_df["Size_MB"] >= 10) &
    (merged_df["Last Updated"].dt.month == 1)
].copy()



merged_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_MB
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333,HEALTH_AND_FITNESS,4.0,2490.0,3.8M,500000,Free,0,Everyone 10+,Health & Fitness,2017-02-17,1.9,2.3.3 and up,3.8
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462,HEALTH_AND_FITNESS,4.0,2490.0,3.8M,500000,Free,0,Everyone 10+,Health & Fitness,2017-02-17,1.9,2.3.3 and up,3.8
2,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000,HEALTH_AND_FITNESS,4.0,2490.0,3.8M,500000,Free,0,Everyone 10+,Health & Fitness,2017-02-17,1.9,2.3.3 and up,3.8
3,10 Best Foods for You,Best idea us,Positive,1.00,0.300000,HEALTH_AND_FITNESS,4.0,2490.0,3.8M,500000,Free,0,Everyone 10+,Health & Fitness,2017-02-17,1.9,2.3.3 and up,3.8
4,10 Best Foods for You,Best way,Positive,1.00,0.300000,HEALTH_AND_FITNESS,4.0,2490.0,3.8M,500000,Free,0,Everyone 10+,Health & Fitness,2017-02-17,1.9,2.3.3 and up,3.8


In [90]:
category_stats = filtered.groupby("Category").agg({
    "Rating": "mean",
    "Reviews": "sum",
    "Installs": "sum"
}).reset_index()

top10 = category_stats.sort_values("Installs", ascending=False).head(10)

In [91]:
sia = SentimentIntensityAnalyzer()
filtered["sentiment"] = filtered["Translated_Review"].apply(
    lambda x: "Positive" if sia.polarity_scores(str(x))["compound"] > 0.05
    else "Negative" if sia.polarity_scores(str(x))["compound"] < -0.05
    else "Neutral"
)

sentiment_counts = filtered["sentiment"].value_counts().reset_index()
sentiment_counts.columns = ["Sentiment", "Count"]



In [92]:
# Ensure Installs and Reviews are numeric
filtered["Installs"] = pd.to_numeric(filtered["Installs"], errors="coerce")
filtered["Reviews"] = pd.to_numeric(filtered["Reviews"], errors="coerce")
filtered["Rating"] = pd.to_numeric(filtered["Rating"], errors="coerce")

# Drop rows with missing values in key columns
filtered = filtered.dropna(subset=["Installs", "Reviews", "Rating"])


In [93]:
# Gradient pastel colors
rating_colors = ['#b2fab4', '#a0e6a0', '#8ce08c', '#79db79', '#66d566',
                 '#53d053', '#40cc40', '#2db92d', '#1ab61a', '#07a507']  # pastel green
reviews_colors = ['#e6c2f0', '#d9aef0', '#cc99f0', '#bf85f0', '#b371f0',
                  '#a85de0', '#9c49d0', '#8f35c0', '#831fb0', '#7610a0']  # pastel purple
trend_colors = ['#ffd8a8', '#ffc680', '#ffb347', '#ffa500', '#ff9933',
                '#ff8c1a', '#ff7f00', '#ff7300', '#ff6600', '#ff5900']  # pastel orange


In [94]:
import plotly.graph_objects as go

categories = top10["Category"]


main_fig = go.Figure()

# Average Rating bar (gradient pastel green)
main_fig.add_trace(go.Bar(
    x=categories,
    y=top10["Rating"],
    name="Average Rating",
    marker=dict(color=rating_colors, line=dict(color='green', width=1)),
    text=top10["Rating"],
    textposition='auto',
    hovertemplate='Category: %{x}<br>Rating: %{y}<extra></extra>'
))

# Total Reviews bar (gradient pastel purple)
main_fig.add_trace(go.Bar(
    x=categories,
    y=top10["Reviews"],
    name="Total Reviews",
    marker=dict(color=reviews_colors, line=dict(color='purple', width=1)),
    text=top10["Reviews"],
    textposition='auto',
    hovertemplate='Category: %{x}<br>Reviews: %{y}<extra></extra>'
))


# Layout customization
main_fig.update_layout(
    title={
        'text': "<b>Top 10 App Categories by Installs</b>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font':dict(family="Georgia, serif", size=18, color="black")
    },
    barmode='group',
    xaxis_title="Category",
    yaxis_title="Values",
    template="plotly_white",  
    height=600,
    width=1000,
    font=dict(family="Garamond, sans-serif", size=12, color="black"),
    margin=dict(l=50, r=50, t=80, b=50),
)


main_fig.show()



In [95]:
import plotly.express as px
import pandas as pd

# Updated pastel color palette
pastel_colors = ["#ffb6c1",  # light pink
                 "#deb887",  # light brown
                 "#add8e6"]  # pastel blue

pastel_sentiments = ["#ffb6c1", "#d8b4ff", "#ffd8a8", "#deb887", "#add8e6"]

# Chart 1: Pie Chart with black outline
chart1 = px.pie(
    filtered,
    names="Category",
    values="Installs",
    title="Install Distribution by Category",
    color="Category",
    color_discrete_sequence=pastel_colors*5
)

chart1.update_traces(marker=dict(line=dict(color="black", width=2)))


# Chart 2: Line Chart (Average Rating per Category) with black markers
avg_rating = filtered.groupby("Category")["Rating"].mean().reset_index()
chart2 = px.line(
    avg_rating,
    x="Category",
    y="Rating",
    title="Average Rating Trend per Category",
    markers=True,
    color_discrete_sequence=["#deb887"]  # light brown line
)
chart2.update_traces(marker=dict(color="#deb887", line=dict(color="black", width=2)))  # black outline

# Chart 3: Scatter Plot with black marker outlines
chart3 = px.scatter(
    filtered,
    x="Reviews",
    y="Rating",
    size="Installs",
    color="Category",
    hover_name="Category",
    title="Reviews vs Rating with Installs",
    color_discrete_sequence=pastel_colors*5
)
chart3.update_traces(marker=dict(line=dict(color="black", width=1.5)))  # black outline

# Chart 4: Bar Chart (Sentiment Analysis) with black outlines
chart4 = px.bar(
    sentiment_counts,
    x="Sentiment",
    y="Count",
    color="Sentiment",
    title="User Sentiment Analysis",
    text="Count",
    color_discrete_sequence=pastel_sentiments
)
chart4.update_traces(marker=dict(line=dict(color="black", width=1.5)))  # black outline

# Update layout for all charts
for chart in [chart1, chart2, chart3, chart4]:
    chart.update_layout(
        template="plotly_white",
        font=dict(family="Garamond, sans-serif", size=12, color="black"),
        title=dict(
            font=dict(family="Georgia, serif", size=18, color="black"),  # title font
            text=f"<b>{chart.layout.title.text}</b>"  # make title bold
        ),
        margin=dict(l=40, r=40, t=60, b=40),
        height=500
    )



In [96]:
html_content = f"""
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>App Analytics Dashboard</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #f9f9f9;
            margin: 0;
            padding: 0;
        }}
        .header {{
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 15px 20px;
            background: white;
            box-shadow: 0px 4px 8px rgba(0,0,0,0.05);
            flex-wrap: wrap;
        }}
        .header h1 {{
            font-size: 24px;
            margin: 10px;
            flex-grow: 1;
            text-align: center;
        }}
        .header img {{
            height: 40px;
        }}
        .clock {{
            font-size: 16px;
            color: #444;
            margin-top: 10px;
            text-align: right;
            width: 100%;
        }}
        .main {{
            text-align: center;
            padding: 20px;
        }}
        .chart-large {{
            margin: auto;
            width: 95%;
            height: 600px;
            border: 2px solid #ddd;
            border-radius: 12px;
            box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
            padding: 10px;
            background: white;
        }}
        .grid {{
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 20px;
            padding: 20px;
        }}
        .chart-small {{
            background: white;
            border-radius: 12px;
            box-shadow: 0px 4px 10px rgba(0,0,0,0.1);
            padding: 10px;
            height:550px;
            height: auto;
        }}    
        #timeMessage {{
            display: none;
            text-align: center;
            font-size: 18px;
            color: #721c24;
            background-color: #f8d7da;
            border: 1px solid #f5c6cb;
            border-radius: 12px;
            padding: 20px;
            margin: 20px auto;
            max-width: 90%;
            box-shadow: 0px 4px 12px rgba(0,0,0,0.1);
            opacity: 0;
            transition: opacity 1s ease-in-out;
        }}
        #timeMessage.show {{
            display: block;
            opacity: 1;
        }}
        /*  Responsive Design */
        @media (max-width: 1024px) {{
            .grid {{
                grid-template-columns: 1fr;
            }}
            .header {{
                flex-direction: column;
                text-align: center;
            }}
            .header img {{
                margin: 5px 0;
            }}
            .clock {{
                position: relative;
                right: auto;
                margin: 10px 0;
                text-align: center;
            }}
        }}
    </style>
</head>
<body>
    <div class="header">
        <img src="https://upload.wikimedia.org/wikipedia/commons/2/2f/Google_2015_logo.svg" alt="Google Logo">
        <h1> App Analytics Dashboard</h1>
        <img src="https://upload.wikimedia.org/wikipedia/commons/7/78/Google_Play_Store_badge_EN.svg" alt="Play Store Logo">
        <div class="clock" id="clock"></div>
    </div>

    <div class="main">
        <div id="mainChartContainer" class="chart-large">
            <div id="mainChartContent">
                {main_fig.to_html(include_plotlyjs=True, full_html=False)}
            </div>
            <div id="timeMessage">
                 This graph will be shown only between <b>3 PM to 5 PM IST</b>.
            </div>
        </div>
    </div>

    <div class="grid">
        <div class="chart-small">{chart1.to_html(include_plotlyjs=False, full_html=False)}</div>
        <div class="chart-small">{chart2.to_html(include_plotlyjs=False, full_html=False)}</div>
        <div class="chart-small">{chart3.to_html(include_plotlyjs=False, full_html=False)}</div>
        <div class="chart-small">{chart4.to_html(include_plotlyjs=False, full_html=False)}</div>
    </div>

    <script>
        function updateClock() {{
            let now = new Date();
            let utc = now.getTime() + (now.getTimezoneOffset() * 60000);
            let ist = new Date(utc + (3600000*5.5));
            document.getElementById("clock").innerHTML = " " + ist.toLocaleTimeString();
        }}
        setInterval(updateClock, 1000);
        updateClock();

        function checkTime() {{
            let now = new Date();
            let utc = now.getTime() + (now.getTimezoneOffset() * 60000);
            let ist = new Date(utc + (3600000*5.5));
            let hour = ist.getHours();

            if (hour < 15 || hour >= 17) {{
                document.getElementById("mainChartContent").style.display = "none";
                document.getElementById("timeMessage").classList.add("show");
            }} else {{
                document.getElementById("mainChartContent").style.display = "block";
                document.getElementById("timeMessage").classList.remove("show");
            }}
        }}
        window.onload = checkTime;
        setInterval(checkTime, 60000);
    </script>
</body>
</html>
"""

with open("interactive.html", "w", encoding="utf-8") as f:
    f.write(html_content)

print("interactive.html responsive website created successfully! Ready to host")


interactive.html responsive website created successfully! Ready to host
